**Рекомендация тарифов**

**Постановка задачи:**

Даны сведения о поведении клиентов, которые уже перешли на тарифы "Смарт" и "Ультра". 

Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не требуется.

**Содержание:**

1. Загрузка требуемых библиотек и датасета
2. Разбиение данных на обучающую и тестовую выборки
3. Создание, обучение и описание моделей
4. Проверка качества полученных моделей на тестовой выборке
5. Проверка адекватности моделей
6. Вывод по проекту

# Загрузка требуемых библиотек и датасета

In [20]:
# Подгружаем требуемые для работы инструменты

import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

import warnings
warnings.simplefilter('ignore')

In [8]:
# Для работы из тренажера или с локальной машины подгружаем датасет с помощью конструкции try-except

try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('users_behavior.csv')

In [9]:
# Осматриваем имеющийся датасет

df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**Выводы по первичному изучению данных**

Датафрейм состоит из 3214 строк и 5 столбцов. Пропусков нет, форматирования данных не требуется.
        
1. сalls — количество звонков;
2. minutes — суммарная длительность звонков в минутах;
3. messages — количество sms-сообщений;
4. mb_used — израсходованный интернет-трафик в Мб;
5. is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).   
</div>

## Вывод по разделу

На данном этапе были подгружены необходимые библиотеки и инструменты для создания, обучения, тестирования ML-моделей. 
Скачан исходный датасет, дано описание его содержимого. Можно двигаться дальше.


# Разбиение данных на выборки

In [11]:
# Создаем датафрейм обучающих признаков

features = df.drop('is_ultra', axis = 1)

In [12]:
# записываем целевой призанак в отдельный датасет

target = df['is_ultra'] 

In [13]:
# Делим исходный датасет на обучающую, валидационную и тестовую выборку в соотношении 3:1:1, для чего используем метод train_test_split два раза 
# со значениями test_size = 0.2 ...

features_eighty, features_valid, target_eighty, target_valid = train_test_split(
    features, target, test_size=0.2, random_state=12345) 

In [14]:
# и 0.25 (сначала исходный датасет делим на 80 и 20 процентов, а затем полученные 80 процентов на 75 и 25)

features_train, features_test, target_train, target_test = train_test_split(
    features_eighty, target_eighty, test_size=0.25, random_state=12345) 

In [15]:
# Смотрим размеры тренировочного набора признаков

features_train.shape

(1928, 4)

In [16]:
# Смотрим размеры валидационного набора признаков

features_valid.shape

(643, 4)

In [17]:
# Смотрим размеры тестового набора признаков

features_test.shape

(643, 4)

## Вывод по разделу

В этом разделе исходный набор объектов был разделен на тренировочную, валидационную и тестовую выборки в соотношении 3:1:1. 
Это позволит нам в дальнейшем обучить модель, исследовать ее точность, а затем проверить результаты обучения.

# Исследование моделей

## Решающее дерево

In [18]:
# Для начала сформируем, обучим модель решающего дерева на тренировочной выборке. Затем рассчитаем метрику accuracy. 
# Все это будем делать для различных гиперпараметров max_depth и min_samples_leaf. 

# Цикл for будет перебирать различные значения глубины дерева
# на отрезке от 1 до 99. Второй цикл для каждой глубины переберет различное количество листьев от 1 до 19.

# В конце каждой итерации цикла полученное значение точности модели 
# будет сравниваться с предыдущим и, в случае его увеличения,
# записываться в переменную best_result. Сама модель - в best_tree,
# ее глубина в best_depth.

best_tree = None
best_result = 0

for depth in range(1, 100):
    for leafs in range(1, 20):
    
        model = DecisionTreeClassifier(random_state=12345, max_depth=depth, min_samples_leaf=leafs) 
        
        model.fit(features_train, target_train) 
        
        predictions = model.predict(features_valid) 
        
        result = accuracy_score(target_valid, predictions) 

        if result > best_result:
            best_tree = model
            best_result = result
            best_depth = depth
            best_leafs = leafs
        
print("Accuracy лучшего дерева:", best_result)
print('Глубина лучшего дерева:', best_depth)
print('Количество листьев лучшего дерева:', best_leafs)

Accuracy лучшего дерева: 0.7962674961119751
Глубина лучшего дерева: 4
Количество листьев лучшего дерева: 4


## Случайный лес

In [19]:
# Следующей моделью для проверки берем случайный лес. Алгоритм действий тот же, что для решающего дерева.

# Цикл for будет перебирать различные значения количества деревьев
# в лесу на отрезке от 1 до 100. 

# В конце каждой итерации цикла полученное значение точности
# модели будет сравниваться с предыдущим и, в случае его увеличения,
# записываться в переменную best_result, сама модель в best_forest,
# а количество деревьев в ней в best_forest_est.

best_model = None
best_result = 0

for est in range(1, 100):
    
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    
    model.fit(features_train, target_train) 
    
    result = model.score(features_valid, target_valid) 
    
    if result > best_result:
        best_forest = model 
        best_result = result
        best_forest_est = est
        
print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print('Количество деревьев в лучшей модели:', best_forest_est)

Accuracy наилучшей модели на валидационной выборке: 0.7931570762052877
Количество деревьев в лучшей модели: 84


## Логистическая регрессия

In [21]:
# Следующей моделью для проверки берем логистическую регрессию. Алгоритм действий тот же, что и для первых двух моделей, но без циклов

model_log_reg = LogisticRegression(random_state=12345) 

model_log_reg.fit(features_train, target_train) 

result = model_log_reg.score(features_valid, target_valid) 

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7589424572317263


# Проверка моделей на тестовой выборке

## Решающее дерево

In [22]:
# Проверим качество предсказаний лучшей модели решающего дерева на тестовой выборке

best_tree.score(features_test, target_test) 

0.7667185069984448

## Случайный лес

In [23]:
# Проверим качество предсказаний лучшей модели решающего дерева на тестовой выборке

best_forest.score(features_test, target_test) 

0.7978227060653188

## Логистическая регрессия

In [17]:
# Проверим качество предсказаний лучшей модели решающего дерева на тестовой выборке

model_log_reg.score(features_test, target_test) 

0.6967340590979783

## Вывод по разделу

На текущем этапе проекта все три полученные модели с наилучшими параметрами были проверены на тестовой выборке. 

Сформированная модель решающего дерева вошла в стадию переобучения, в связи с чем показатель ее точности на тестовой выборке упал, по-сравнению с валидационной (0.796 против 0.766). 

Случайный лес показал себя с лучшей стороны. Его показатель accuracy на валидационной выборке оказался даже слегка ниже, чем на тестовой (0.793 против 0.797).

Логистическая регрессия оказалась в состоянии недообучения - результаты на валидационной выборке не могли похвастаться высоким показателем точности, а на тестовой совсем скатились вниз (0.702 против 0.696)

## (бонус) Проверьте модели на адекватность

In [18]:
# А проверку на адекватность будем делать с помощью инструмента Dummy Models (спасибо за наводку на этот метод, Алексей!) 
# При этом для широты эксперимента попробуем применить метод формирования предсказаний по самому частотному параметру (most_frequent),
# c учетом распределения по классам (stratified), при равномерном распределении вероятности прогноза (uniform). Стратегию предсказания по константам
# можно не рассматривать, так как она будет частным случаем для самого частотного параметра.

strategies = ['most_frequent', 'stratified', 'uniform']
  
scores = []
for strat in strategies:
    model = DummyClassifier(strategy = strat, random_state = 0)
    model.fit(features_train, target_train)
    score = model.score(features_test, target_test)
    print('Показатель точности для стратегии {}: {}'.format(strat, score) )


Показатель точности для стратегии most_frequent: 0.6889580093312597
Показатель точности для стратегии stratified: 0.5909797822706065
Показатель точности для стратегии uniform: 0.48833592534992226


## Вывод по проверке модели на адекватность
    
В данном разделе модель DummyClassifier при разных параметрах стратегий (most_frequent, stratified, uniform) была обучена на тренировочной выборке, а затем ее работоспособность проверена на тестовой выборке. 

Результаты продемонстрировали, что все три стратегии оказались "слабее" по значению accuracy, чем любая из наших исследованных ранее моделей. Только показатели логистической регрессии немногим превзошли лучший показатель DummyClassifier (0.6967340590979783 против 0.6889580093312597). Таким образом можно уверенно заявить, что наши модели оказались более эффективными, чем простейший классификатор. 
    С этим можно работать.  


</div>

# Вывод по проекту
   
В данном проекте были отработаны навыки загрузки необходимых для машинного обучения библиотек, формирования моделей классификации и перебора их гиперпараметров. Проверка качества модели на валидационной выборке, а затем на тестовой.

Для данного датасета **лучше всего себя показала модель случайного леса (accuracy = 0.798)**, показывая одинаково хорошие результаты на обеих проверках. Несколько хуже отработало решающее дерево (accuracy = 0.767), а в самом конце плелась логистическая регрессия (accuracy = 0.697).

Таким образом, для решения задачи по выбору подходящего тарифа для клиента следовало бы брать модель случайного леса.

В заключительном разделе была проведена проверка моделей на адекватность путем сравнения их показателей Accuracy  с аналогичной метрикой инструмента DummyClassifier. Результаты показали, что полученные нами модели оказались эффективнее проверочной системы.  